## Training file
This file runs training for a given file

In [5]:
import torch
import numpy as np
from ipynb.fs.full.Utils import arrhenius
from ipynb.fs.full.Data_loader import map_to_scale
import pandas as pd

In [6]:
def train_model(model,optimizer,criterion,n_epochs=100,weight=[1,1,1,1],bweight=[1,1,1,1],train_loader=None, scheduler=None, valid_loader=None, scaler=None):
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    losses = {'train':[],'valid':[]}
    valid_loss_min = np.Inf
    train_loss_min = np.Inf
    
    try:
        model.fck # check if model outputs k, if not skip to Arrhenius loss
        
        for epoch in range(1, n_epochs+1):

            # keep track of training and validation loss
            train_loss = 0.0
            valid_loss = 0.0
            ###################
            # train the model #
            ###################
            model.train()
            for features, target, temp in train_loader:
                features = features.to(device)
                target = target.to(device)
                temp   = temp.to(device)
                temp = temp.reshape([temp.shape[0],1]);
                if model.fck.out_features==1:
                    targetlogk=target[:,0].reshape([target.shape[0],1])
                else:
                    targetlogk=target[:,[0,4,8,12]]
                data = torch.cat((features,temp),1)
                optimizer.zero_grad()
                # forward pass: compute predicted outputs by passing inputs to the model
                logk = model(data)
                # calculate the batch loss
                loss = calculate_loss_simple(targetlogk,logk,criterion)
                # backward pass: compute gradient of the loss with respect to model parameters
                loss.backward()
                # perform a single optimization step (parameter update)
                optimizer.step()    
                # update training loss
                train_loss += loss.item()*data.size(0)
            
            model.eval()
            for features, target, temp in valid_loader:
                features = features.to(device)
                target = target.to(device)
                temp   = temp.to(device)
                temp = temp.reshape([temp.shape[0],1])
                if model.fck.out_features==1:
                    targetlogk=target[:,0].reshape([target.shape[0],1])
                else:
                    targetlogk=target[:,[0,4,8,12]]
                data = torch.cat((features,temp),1)
                # forward pass: compute predicted outputs by passing inputs to the model
                logk = model(data)
                # calculate the batch loss
#                 loss = calculate_loss(scaler.torch_inverse(targetlogk),scaler.torch_inverse(logk),criterion,bweight)
                loss = calculate_loss_simple(targetlogk,logk,criterion)
                valid_loss += loss.item()*data.size(0)
                
            scheduler.step()
            # calculate average losses
            train_loss = train_loss/len(train_loader.dataset)
            losses['train'].append(train_loss)
            valid_loss = valid_loss/len(valid_loader.dataset)
            losses['valid'].append(valid_loss)
            # print training statistics 
            if epoch%10 == 0 or epoch ==1:
                print('Epoch: {} \tTraining Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                    epoch, train_loss, np.sqrt(train_loss)))
                if not (valid_loader==None):
                    print('\t\tValidation Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                            valid_loss, np.sqrt(valid_loss)))    
        
    except AttributeError :

        for epoch in range(1, n_epochs+1):

            # keep track of training and validation loss
            train_loss = 0.0
            train_lossA = 0.0
            train_lossB = 0.0
            train_lossn = 0.0
            train_lossArrhenius = 0.0
            valid_loss = 0.0
            valid_lossArrhenius = 0.0
            rmse_loss = 0.0
            total_notnan = 0

            
            ###################
            # train the model #
            ###################
            model.train()
            for features, target, temp in train_loader:
                features = features.to(device)
                target = target.to(device)
                temp   = temp.to(device)
                optimizer.zero_grad()
                # forward pass: compute predicted outputs by passing inputs to the mode
                A,n,B = model(features)
                # calculate the batch loss
                lossA,lossn,lossB,lossArrhenius = calculate_arrhenius_loss(target,A,B,n,criterion,temp,model,scaler,bweight)
                loss = weight[0]*lossArrhenius+weight[1]*lossA+weight[2]*lossn+weight[3]*lossB 
                # backward pass: compute gradient of the loss with respect to model parameters
                loss.backward()
                # perform a single optimization step (parameter update)
                optimizer.step()    
                # update training loss
                train_lossA += lossA.item()*features.size(0)
                train_lossB += lossB.item()*features.size(0)
                train_lossn += lossn.item()*features.size(0)
                train_lossArrhenius += lossArrhenius.item()*features.size(0)
                train_loss += loss.item()*features.size(0)

            model.eval()
            for features, target, temp in valid_loader:
                features = features.to(device)
                target = target.to(device)
                temp   = temp.to(device)
                # forward pass: compute predicted outputs by passing inputs to the mode
                A,n,B = model(features)
                lossA,lossn,lossB,lossArrhenius = calculate_arrhenius_loss(target,A,B,n,criterion,temp,model,scaler,bweight)
                loss = weight[0]*lossArrhenius+weight[1]*lossA+weight[2]*lossn+weight[3]*lossB
                # update training loss
                valid_loss += loss.item()*features.size(0)
                valid_lossArrhenius += lossArrhenius.item()*features.size(0)
#                 rmse,notnan = calculate_rmse(target,A,B,n,criterion,temp,model,scaler,test_on)
#                 total_notnan += notnan
#                 rmse_loss += rmse.item()
                    
            scheduler.step()
            # calculate average losses
            train_loss = train_loss/len(train_loader.dataset)
            train_lossA = train_lossA/len(train_loader.dataset)
            train_lossB = train_lossB/len(train_loader.dataset)
            train_lossn = train_lossn/len(train_loader.dataset)
            train_lossArrhenius = train_lossArrhenius/len(train_loader.dataset)
            valid_lossArrhenius = valid_lossArrhenius/len(valid_loader.dataset)
#             rmse_loss = (rmse_loss/total_notnan)
            losses['train'].append(train_loss)
            #losses['trainA'].append(train_lossA)
            #losses['trainB'].append(train_lossB)
            #losses['trainn'].append(train_lossn)
            #losses['trainArrhenius'].append(train_lossArrhenius)
            valid_loss = valid_loss/len(valid_loader.dataset)
            losses['valid'].append(valid_loss)
            # print training statistics
            if epoch%10 == 0 or epoch ==1:
                print('Epoch: {} \tTraining Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                    epoch, train_loss, np.sqrt(train_loss)))
                print('Epoch: {} \tTraining A Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                    epoch, train_lossA, np.sqrt(train_lossA)))
                print('Epoch: {} \tTraining B Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                    epoch, train_lossB, np.sqrt(train_lossB)))
                print('Epoch: {} \tTraining n Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                    epoch, train_lossn, np.sqrt(train_lossn)))
                print('Epoch: {} \tTraining Arrhenius Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                    epoch, train_lossArrhenius, np.sqrt(train_lossArrhenius)))
                print('Epoch: {} \tTraining Arrhenius Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
                    epoch, valid_lossArrhenius, np.sqrt(valid_lossArrhenius)))
#                 print('Epoch: {} \tTraining Arrhenius Loss: MSE = {:.6f} ; RMSE = {:.6f}'.format(
#                     epoch, rmse_loss, np.sqrt(rmse_loss)))
    
    return(losses)

In [7]:

def calculate_loss_simple(target,output,criterion):
    loss = torch.nan_to_num(criterion(output[~target[:,0].isnan(),0],target[~target[:,0].isnan(),0]))
    for t in range(1,output.shape[1]):
        loss += torch.nan_to_num(criterion(output[~target[:,t].isnan(),t],target[~target[:,t].isnan(),t]))
    return loss


def calculate_loss(target,output,criterion,bweight):
#     loss = bweight[0]*torch.nan_to_num(criterion(output[~target[:,0].isnan(),0],target[~target[:,0].isnan(),0]))
#     for t in range(1,output.shape[1]):
#         loss += bweight[t]*torch.nan_to_num(criterion(output[~target[:,t].isnan(),t],target[~target[:,t].isnan(),t]))
#     return loss
    loss = bweight[0]*torch.nan_to_num(criterion(output[~target[:,0].isnan(),0],target[~target[:,0].isnan(),0])/target[~target[:,0].isnan(),0].shape[0])
    for t in range(1,output.shape[1]):
        loss += bweight[t]*torch.nan_to_num(criterion(output[~target[:,t].isnan(),t],target[~target[:,t].isnan(),t])/target[~target[:,t].isnan(),t].shape[0])
    return loss

def calculate_arrhenius_loss(target,A,B,n,criterion,temp,model,scaler,bweight):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Map target to variables
    j = np.array(list(range(model.fcA.out_features)))*4
    targetlogk = target[:,j]
    targetA = target[:,(j+1)]
    targetB = target[:,(j+2)]
    targetn = target[:,(j+3)]
    # Calculate loss for each variable
    lossA = calculate_loss(targetA,A,criterion,bweight)
    lossn = calculate_loss(targetn,n,criterion,bweight)
    lossB = calculate_loss(targetB,B,criterion,bweight)
    # Scale transform if needed
    if scaler.tscaler != None:
        scaled = scaler.torch_inverse(map_to_scale(A,B,n))
        j = np.array(list(range(model.fcA.out_features)))*3
        A = scaled[:,(j+0)]
        B = scaled[:,(j+1)]
        n = scaled[:,(j+2)]
    # Calculate Arrhenius loss
#     lossArrhenius = bweight[0]*torch.nan_to_num(criterion(scaler.torch_transform(0,arrhenius(torch.pow(10.,A[:,0]),n[:,0],B[:,0],temp))[~targetlogk[:,0].isnan()],targetlogk[~targetlogk[:,0].isnan(),0])/targetlogk[~targetlogk[:,0].isnan(),0].shape[0])
#     for t in range(1,A.shape[1]):
#         lossArrhenius += bweight[t]*torch.nan_to_num(criterion(scaler.torch_transform(t,arrhenius(torch.pow(10.,A[:,t]),n[:,t],B[:,t],temp))[~targetlogk[:,t].isnan()], targetlogk[~targetlogk[:,t].isnan(),t])/targetlogk[~targetlogk[:,t].isnan(),t].shape[0])
#     return(lossA,lossn,lossB,lossArrhenius)
    lossArrhenius = bweight[0]*torch.nan_to_num(criterion(arrhenius(torch.pow(10.,A[:,0]),n[:,0],B[:,0],temp)[~targetlogk[:,0].isnan()], targetlogk[~targetlogk[:,0].isnan(),0])/targetlogk[~targetlogk[:,0].isnan(),0].shape[0])
    for t in range(1,A.shape[1]):
        lossArrhenius += bweight[t]*torch.nan_to_num(criterion(arrhenius(torch.pow(10.,A[:,t]),n[:,t],B[:,t],temp)[~targetlogk[:,t].isnan()], targetlogk[~targetlogk[:,t].isnan(),t])/targetlogk[~targetlogk[:,t].isnan(),t].shape[0])
    return(lossA,lossn,lossB,lossArrhenius)

In [ ]:
def calculate_rmse(target,A,B,n,criterion,temp,model,scaler,test_on):
    bweight=[1,1,1,1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Map target to variables
    j = np.array(list(range(model.fcA.out_features)))*4
    targetlogk = target[:,j]
    targetA = target[:,(j+1)]
    targetB = target[:,(j+2)]
    targetn = target[:,(j+3)]
    # Calculate loss for each variable
    # Scale transform if needed
    if scaler.tscaler != None:
        scaled = scaler.torch_inverse(map_to_scale(A,B,n))
        j = np.array(list(range(model.fcA.out_features)))*3
        A = scaled[:,(j+0)]
        B = scaled[:,(j+1)]
        n = scaled[:,(j+2)]
    # Calculate Arrhenius loss
#     lossArrhenius = bweight[0]*torch.nan_to_num(criterion(scaler.torch_transform(0,arrhenius(torch.pow(10.,A[:,0]),n[:,0],B[:,0],temp)[~targetlogk[:,0].isnan()]),targetlogk[~targetlogk[:,0].isnan(),0]))
#     for t in range(1,A.shape[1]):
#         lossArrhenius += bweight[t]*torch.nan_to_num(criterion(scaler.torch_transform(t,arrhenius(torch.pow(10.,A[:,t]),n[:,t],B[:,t],temp)[~targetlogk[:,t].isnan()]), targetlogk[~targetlogk[:,t].isnan(),t]))
#     lossArrhenius = bweight[0]*torch.nan_to_num(criterion(arrhenius(torch.pow(10.,A[:,0]),n[:,0],B[:,0],temp)[~targetlogk[:,0].isnan()],targetlogk[~targetlogk[:,0].isnan(),0]))
#     for t in range(1,A.shape[1]):
#         lossArrhenius += bweight[t]*torch.nan_to_num(criterion(arrhenius(torch.pow(10.,A[:,t]),n[:,t],B[:,t],temp)[~targetlogk[:,t].isnan()], targetlogk[~targetlogk[:,t].isnan(),t]))
    print(model.fcA.out_features,test_on)
    if model.fcA.out_features==4 and test_on=='OH':
        t=0
    elif model.fcA.out_features==4 and test_on=='O3':
        t=1
    elif model.fcA.out_features==4 and test_on=='NO3':
        t=2
    elif model.fcA.out_features==4 and test_on=='Cl':
        t=3
    else:
        t=0
    
    print(t)
#     lossArrhenius = torch.nan_to_num(criterion(arrhenius(torch.pow(10.,A[:,t]),n[:,t],B[:,t],temp)[~targetlogk[:,t].isnan()],scaler.torch_inverse2(t,targetlogk[~targetlogk[:,t].isnan(),t]))*targetlogk[~targetlogk[:,t].isnan(),t].shape[0])
    lossArrhenius = torch.nan_to_num(criterion(arrhenius(torch.pow(10.,A[:,t]),n[:,t],B[:,t],temp)[~targetlogk[:,t].isnan()],targetlogk[~targetlogk[:,t].isnan(),t])*targetlogk[~targetlogk[:,t].isnan(),t].shape[0])
    return lossArrhenius,targetlogk[~targetlogk[:,t].isnan(),t].shape[0]